## Topics covered
1. Use free data provider 12 Data to make a 5 min interval dataframe
2. Use TA-Lib to get engulfing data
3. Provide all the parameters required
4. Paper trading simulator
5. Backtesting
6. Analysis of backtesting

### Backtesting Analysis
1. Return analysis
2. Risk adjusted return (Sharepe ratio)
3. Risk Analysis
    1. Volatility
    2. Value at risk
4. Excess returns
5. Confidence interval for one candle
6. Maximum drawdown analysis

Core Satellite approach
---
if the outlook is positive: 80% + 20% = 100% will be invested into Bitcoin <br>
if the outlook is negative: 80% - 20% = 60% will be invested into Bitcoin <br>

Trading exposure on BTC: From 60% to 100%<br>

The main idea behind this is to reduce the volitility, and try to capture the good oppurtunity as well.

Trade interval: 5 min<br>
Amount: $1000<br>
No transaction cost is considered<br>

# Part 1
Making dataframe and using TA-Lib to get engulfing data


In [3]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
import warnings

In [4]:
warnings.filterwarnings('ignore')

In [5]:

API_KEY = "90fe71ead2844dde9376db366bab4ad1"

symbol = "BTC/USD"
interval = "5min"
order = 'asc'


In [6]:
# will be used later
start_date1 = dt.datetime(2021, 7, 4)

# end date is set as the current date
end_date = start_date1

# timedelta is used to subtract the days from the end date
start_date = end_date - dt.timedelta(days=15)

start_date, end_date


(datetime.datetime(2021, 6, 19, 0, 0), datetime.datetime(2021, 7, 4, 0, 0))

12 Data has a limitation to fetch data at one request.
So we send multiple request using a loop and combine them into a final dataset.

In [7]:
api_url_predicted = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/{interval}/{start_date}/{end_date}?unadjusted=true&sort={order}&limit=1200&apiKey={API_KEY}"

api_url = f'https://api.twelvedata.com/time_series?symbol={symbol}&start_date={start_date}&end_date={end_date}&interval={interval}&order={order}&apikey={API_KEY}'

In [8]:
data = requests.get(api_url).json()
data.keys()


dict_keys(['meta', 'values', 'status'])

In [9]:
data_final = pd.DataFrame(data['values'])
data_final

,datetime,open,high,low,close
0,2021-06-19 00:00:00,35835.73047,35835.73047,35681.19141,35736.05078
1,2021-06-19 00:05:00,35742.08984,35851.03125,35726.46094,35851.03125
2,2021-06-19 00:10:00,35844.51953,35937.78125,35812.50000,35937.78125
3,2021-06-19 00:15:00,35941.37109,35941.37109,35820.75000,35862.17188
4,2021-06-19 00:20:00,35858.62109,35858.62109,35786.37109,35789.16016
...,...,...,...,...,...
4268,2021-07-03 19:40:00,34688.85938,34714.14844,34681.17188,34700.07812
4269,2021-07-03 19:45:00,34701.51953,34711.48828,34676.16016,34707.76172
4270,2021-07-03 19:50:00,34707.32812,34786.08984,34704.14062,34773.80078
4271,2021-07-03 19:55:00,34773.05078,34777.37109,34751.23047,34755.01172


In [10]:
# n times 15 days data will be fetched
n = 4

final_df = pd.DataFrame()

for i in range(n):
    # we should append the data in the reverse order, so i is decrementing each time the loop runs
    i = n - i - 1

    end_date = start_date1 - dt.timedelta(days=15 * i)
    start_date = end_date - dt.timedelta(days=15)

    api_url = f'https://api.twelvedata.com/time_series?symbol={symbol}&start_date={start_date}&end_date={end_date}&interval={interval}&order={order}&apikey={API_KEY}'
    data_json = requests.get(api_url).json()
    data_df = pd.DataFrame(data['values'])
    final_df = pd.concat([final_df, data_df[:-1]], ignore_index=True)

final_df


,datetime,open,high,low,close
0,2021-06-19 00:00:00,35835.73047,35835.73047,35681.19141,35736.05078
1,2021-06-19 00:05:00,35742.08984,35851.03125,35726.46094,35851.03125
2,2021-06-19 00:10:00,35844.51953,35937.78125,35812.50000,35937.78125
3,2021-06-19 00:15:00,35941.37109,35941.37109,35820.75000,35862.17188
4,2021-06-19 00:20:00,35858.62109,35858.62109,35786.37109,35789.16016
...,...,...,...,...,...
17083,2021-07-03 19:35:00,34688.89062,34699.00000,34655.19922,34690.92969
17084,2021-07-03 19:40:00,34688.85938,34714.14844,34681.17188,34700.07812
17085,2021-07-03 19:45:00,34701.51953,34711.48828,34676.16016,34707.76172
17086,2021-07-03 19:50:00,34707.32812,34786.08984,34704.14062,34773.80078


In [13]:
from talib import abstract

engulfing = abstract.CDLENGULFING(final_df)
final_df["Engulfing"] = engulfing
final_df[final_df["Engulfing"] != 0]


,datetime,open,high,low,close,Engulfing
7,2021-06-19 00:35:00,35808.75000,35884.73047,35788.26172,35852.87891,100
21,2021-06-19 01:45:00,36033.08984,36141.44922,36033.08984,36141.44922,100
30,2021-06-19 02:30:00,35956.94141,35956.94141,35704.76172,35704.76172,-100
36,2021-06-19 03:00:00,35521.62109,35567.01172,35426.57812,35458.67188,-100
49,2021-06-19 04:05:00,35090.32031,35156.30078,35052.39062,35156.30078,100
...,...,...,...,...,...,...
17056,2021-07-03 17:20:00,34572.85938,34639.69141,34572.85938,34635.00000,100
17059,2021-07-03 17:35:00,34608.96094,34704.85156,34608.96094,34703.76953,100
17068,2021-07-03 18:20:00,34669.12109,34669.12109,34629.28906,34646.80078,-100
17075,2021-07-03 18:55:00,34721.78906,34733.46094,34663.05078,34663.05078,-100


# Part 2
Backtesting


# Part 3
Analysis of backtesting


# Part 4
Drawdown analysis